In [155]:
import json
import csv
from statistics import mean

# 1. **Leitura de Dados:**
def ler_arquivo(arquivo_json):
    try:
        # O with  garante que o arquivo seja fechado automaticamente 
        with open(arquivo_json, 'r') as arquivo:  
            dados = json.load(arquivo)
            return dados
    except FileNotFoundError:
        print(f"Arquivo '{arquivo_json}' não encontrado.")
        return None
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {str(e)}")
        return None

# Função para salvar dados em um arquivo JSON
def salvar_arquivo(dados, arquivo_json):
    try:
        # Obtém a representação em formato de string dos dados serializados em JSON
        dados_json_string = json.dumps(dados, indent=2)
        
        # Salva a string JSON no arquivo
        with open(arquivo_json, 'w') as arquivo:
            arquivo.write(dados_json_string)
            
    except Exception as e:
        raise Exception(f"Erro ao salvar JSON: {str(e)}")



In [156]:
# 2. **Manipulação de Dados:**
def mapeamento(dados):
    # Para cada dicionário x na lista dados, retorna um novo dicionário contendo apenas as chaves 'nome' e 'idade'.
    return list(map(lambda x: {'nome': x['nome'], 'idade': x['idade']}, dados))
# Função para filtrar dados por área de atuação
def filtro(dados, area_atuacao):
    return list(filter(lambda x: x['area_atuacao'] == area_atuacao, dados))
# Função para realizar a soma de salários
def reducao(dados):
    return sum([p['salario'] for p in dados])


# 3. **CRUD (Create, Read, Update, Delete):**
# Função para criar  dados 
def criar_pessoa(dados, arquivo_json, **kwargs):
    try:
        # Construir o novo registro usando as chaves fornecidas
        novo_registro = {key: kwargs.get(key) for key in ['nome', 'idade', 'sexo', 'empregado', 'area_atuacao', 'salario']}
        # Adiciona o novo registro ao final
        dados.append(novo_registro)
        # Obtém a representação em formato de string dos dados serializados em JSON
        dados_json_string = json.dumps(dados, indent=2)
        
        # Salva a string JSON atualizada no arquivo JSON
        with open(arquivo_json, 'w') as arquivo:
            arquivo.write(dados_json_string)
            
        print("Registro salvo com sucesso!")
        
    except Exception as e:
        raise Exception(f"Erro ao salvar registro: {str(e)}")
    
# Função para ler  dados  de uma pessoa    
def ler_pessoa(dados, nome):
    pessoa = next((p for p in dados if p['nome'] == nome), None)
    if pessoa:
        return pessoa
    else:
        raise Exception(f"Pessoa com nome '{nome}' não encontrada.")

# Função para atualizar os dados
def atualizar_pessoa(dados, nome, **kwargs):
    pessoa = ler_pessoa(dados, nome)
    
    for chave, novo_valor in kwargs.items():
        # Verifica se a chave existe no dicionário
        if chave in pessoa:
            pessoa[chave] = novo_valor
            salvar_arquivo(dados, 'dados.json')
            print(f"{chave} atualizado para {novo_valor} para a pessoa {nome}")
        else:
            print(f"A chave {chave} não existe para a pessoa {nome}")
            
# Função para deletar uma pessoa 
def deletar_pessoa(dados, nome):
    pessoa = ler_pessoa(dados, nome)
    dados.remove(pessoa)
    salvar_arquivo(dados, 'dados.json')
    


In [157]:

# 5. **Estatísticas Simples:**

# Função para obter a quantidade de pessoas por área
def qtd_pessoas(dados, area_desejada=None):
    # Filtrar para uma área específica se fornecida
    if area_desejada:
        dados = list(filter(lambda x: x['area_atuacao'] == area_desejada, dados))
    # Iterar sobre as áreas únicas presentes nos dados
    areas_unicas = set([pessoa['area_atuacao'] for pessoa in dados if pessoa['area_atuacao']])
    for area in areas_unicas:
        # Filtrar pessoas apenas para a área específica
        pessoas_area = list(filter(lambda x: x['area_atuacao'] == area, dados))
        # Contar a quantidade de pessoas na área
        quantidade = len(pessoas_area)
        
    
    return quantidade

# Média de idade
def calcular_media_idade(dados, area_atuacao): # Função para obter a média de idade conforme área de atuação
    pessoas_area = list(filter(lambda x: x['area_atuacao'] == area_atuacao, dados))
    idades = [pessoa['idade'] for pessoa in pessoas_area]
    media = mean(idades)
    media_arredondada = round(media, 2)
    return media_arredondada

# 6. **Obtenção de Máximo ou Mínimo:**
def max_min(dados, atributo, maximo=True):
    extremos = [(pessoa['nome'], pessoa[atributo]) for pessoa in dados]
    extremo = max(extremos, key=lambda x: x[1]) if maximo else min(extremos, key=lambda x: x[1])
    return extremo

def max_min_por_area(dados, atributo, area_atuacao, maximo=True):
    extremos = [(pessoa['nome'], pessoa[atributo]) for pessoa in dados if pessoa['area_atuacao'] == area_atuacao]
    
    if not extremos:
        return None  # Retorna None se não houver pessoas na área de atuação especificada
    
    extremo = max(extremos, key=lambda x: x[1]) if maximo else min(extremos, key=lambda x: x[1])
    return extremo

# Função para calcular estatísticas 
def estatisticas_por_area(dados, area_atuacao):
    media_idade = calcular_media_idade(dados, area_atuacao)
    max_nome, max_salario = max_min_por_area(dados, "salario", area_atuacao,maximo=True)
    min_nome, min_salario = max_min_por_area(dados, "salario",area_atuacao, maximo=False)
    qtd =  qtd_pessoas(dados,area_atuacao)
    return {
        "area_atuacao": area_atuacao,
        "qtd_pessoas": qtd,
        "media_idade": media_idade,
        "max_salario_nome": max_nome,
        "max_salario": max_salario,
        "min_salario_nome": min_nome,
        "min_salario": min_salario
    }
    
    
    
# Função para exportar estatísticas para um arquivo CSV
def exportar_csv(estatisticas, arquivo_csv):
    try:
        with open(arquivo_csv, mode='w', newline='') as arquivo:
            writer = csv.DictWriter(arquivo, fieldnames=estatisticas.keys())
            writer.writeheader()
            writer.writerow(estatisticas)
    except Exception as e:
        raise Exception(f"Erro ao exportar CSV: {str(e)}")


    
   

In [158]:
def salvar_csv(estatisticas_por_area, arquivo_csv):
    try:
        with open(arquivo_csv, mode='w', newline='') as arquivo:
            writer = csv.writer(arquivo, delimiter=";",lineterminator='\n')
            # Escreve o cabeçalho
            writer.writerow(["Area Atuacao", "Qtd Pessoas", "Media Idade", "Max Nome", "Max Salario", "Min Nome", "Min Salario"])

            # Escreve os dados
            for estatisticas in estatisticas_por_area:
                writer.writerow([estatisticas["area_atuacao"],
                                 estatisticas["qtd_pessoas"],
                                 estatisticas["media_idade"],
                                 estatisticas["max_salario_nome"],
                                 estatisticas["max_salario"],
                                 estatisticas["min_salario_nome"],
                                 estatisticas["min_salario"]])
                print(estatisticas)
            
        print(f"Dados exportados para '{arquivo_csv}' com sucesso.")
    except Exception as e:
        print(f"Erro ao exportar CSV: {str(e)}")

# Função para calcular estatísticas para todas as áreas
def gerar_estatisticas(dados):
    lista_estatisticas = []
    # Obtém todas as áreas únicas de atuação
    areas_de_atuacao = set(dado['area_atuacao'] for dado in dados)
   
    for area_atuacao in areas_de_atuacao:
        estatisticas = estatisticas_por_area(dados, area_atuacao)
        lista_estatisticas.append(estatisticas)

    return lista_estatisticas

In [159]:
def exibir_menu():
    print("1. Criar registro")
    print("2. Atualizar registro")
    print("3. Deletar registro")
    print("4. Gerar estatísticas")
    print("5. Sair")
    print("\n")
    
    opcao = input("Escolha uma opção (1-5): ")
    
    return opcao

def interacao_usuario(dados):
    while True:
        opcao = exibir_menu()

        if opcao == '1':
            nome = input("Digite o nome da pessoa: ")
            idade = int(input("Digite a idade: "))
            sexo = input("Digite o sexo: ")
            area = input("Digite a área de atuação: ")
            salario = float(input("Digite o salario: "))
            
            arquivo_json = 'dados.json'
            try:
               criar_pessoa(dados, arquivo_json, nome=nome, idade=idade, sexo=sexo, empregado=False, area_atuacao=area, salario=salario)
               print("Registro atualizado com sucesso.")
            except Exception as e:
                print(str(e))

        elif opcao == '2':
            nome = input("Digite o nome da pessoa para atualizar o salário: ")
            
            try:
                atualizar_pessoa(dados, nome, idade=30, salario=50000, area_atuacao='TI')
                print("Registro atualizado com sucesso.")
            except Exception as e:
                print(str(e))
                
        elif opcao == '3':
            nome = input("Digite o nome da pessoa para deletar o registro: ")
            try:
                deletar_pessoa(dados, nome)
                print("Registro deletado com sucesso.")
            except Exception as e:
                print(str(e))
        
                      
        elif opcao == '4':
            try:
                arquivo_csv = 'estatisticas.csv'
                estatisticas = gerar_estatisticas(dados)
                salvar_csv(estatisticas, arquivo_csv)
                
                print(f"\nDados estatísticos exportados para '{arquivo_csv}'.")
                
            except Exception as e:
                print(str(e))

        elif opcao == '5':
            print("Encerrando o programa.")
            break

        else:
            print("Opção inválida. Por favor, escolha novamente.")


    
    

In [160]:
arquivo_json = 'dados.json'
dados = ler_arquivo(arquivo_json)
interacao_usuario(dados)

1. Criar registro
2. Atualizar registro
3. Deletar registro
4. Gerar estatísticas
5. Sair


Registro salvo com sucesso!
Registro atualizado com sucesso.
1. Criar registro
2. Atualizar registro
3. Deletar registro
4. Gerar estatísticas
5. Sair


Registro deletado com sucesso.
1. Criar registro
2. Atualizar registro
3. Deletar registro
4. Gerar estatísticas
5. Sair


Encerrando o programa.


In [161]:


if dados is not None:
    # Continue com o processamento dos dados
    print("Dados carregados com sucesso.")
    #interacao_usuario(dados)
    arquivo_csv = 'estatisticas.csv'
    estatisticas_area = gerar_estatisticas(dados)
    salvar_csv(estatisticas_area, arquivo_csv)
                
    print(f"\nDados estatísticos exportados para '{arquivo_csv}'.")
else:
    # Trate o caso em que o arquivo não foi encontrado ou houve um erro de decodificação JSON
    print("Falha ao carregar dados.")

Dados carregados com sucesso.
{'area_atuacao': 'Educacao', 'qtd_pessoas': 76, 'media_idade': 40.99, 'max_salario_nome': 'Rafael Silva', 'max_salario': 9835.51, 'min_salario_nome': 'Camila Costa', 'min_salario': 3030.4}
{'area_atuacao': 'Engenharia', 'qtd_pessoas': 70, 'media_idade': 41.46, 'max_salario_nome': 'Ana Souza', 'max_salario': 9973.85, 'min_salario_nome': 'Lucas Almeida', 'min_salario': 3010.92}
{'area_atuacao': 'Financas', 'qtd_pessoas': 75, 'media_idade': 39.27, 'max_salario_nome': 'Rafael Pereira', 'max_salario': 9862.6, 'min_salario_nome': 'Lucas Lima', 'min_salario': 3111.37}
{'area_atuacao': 'Administracao', 'qtd_pessoas': 72, 'media_idade': 40.51, 'max_salario_nome': 'Camila Costa', 'max_salario': 9923.61, 'min_salario_nome': 'Rafael Almeida', 'min_salario': 3094.83}
{'area_atuacao': 'Saude', 'qtd_pessoas': 84, 'media_idade': 39.35, 'max_salario_nome': 'Camila Almeida', 'max_salario': 9807.2, 'min_salario_nome': 'Ana', 'min_salario': 2500.0}
{'area_atuacao': 'Nenhuma',